In [3]:
import nltk
import re
nltk.download('punkt')
from nltk.tokenize import RegexpTokenizer, sent_tokenize
import word2number
import num2words

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\binar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [128]:
def readFile(filename):
    """
    This function reads the file given the filename and returns the contents
    """
    with open(filename, 'r', encoding='utf8') as f:
        return f.read()

def getMetaData(text):
    """
    I observed a pattern of metatags on the top of file which is defined by the regex given below
    This function given the entire text of the file separates the meta data from the main text in the file
    and returns them separately
    """
    pattern = re.compile(r'([a-zA-Z0-9-]+: .+[\n][\n])')
    metaData = pattern.split(text, 1)
    metaData[0] = metaData[0] + metaData[1]
    return metaData[0], metaData[2]
    
def getSentences(text):
    """
    The data that we have received in the files is not all properly structured
    1. To handle all the cases, we first use PunktSentenceTokenizer from the NLTK library
    2. On top of it we use double or more \n to separate sentences
    3. The sentences are then stripped of whitespaces to look for any sentence that is just blank
    4. Finally we remove those sentences and return the remaining sentences as the valid ones 
    """
    meta, data = getMetaData(text)
    tokenizer = nltk.tokenize.punkt.PunktSentenceTokenizer()
    sentences = tokenizer.tokenize(data)
    for s in range(len(sentences)):
        splitSentence = re.split('^[^a-zA-Z0-9. \n]{2,}|[\n]{2,}', sentences[s])
        sentences[s] = splitSentence[0]
        i = 1
        while i < len(splitSentence):
            sentences.insert(s+i, splitSentence[i].strip())
            i += 1
        s += len(splitSentence)
    s = 0
    while s < (len(sentences)):
        if (len(sentences[s].strip()) == 0):
            sentences.pop(s)
            continue
        sentences[s] = sentences[s].strip()
        s += 1
    return sentences

def getWords(text):
    """
    Here we observed another couple of cases
    1. Some words like emails, dates and others could be separated by using NLTKs word tokenizer
    2. So we employed 3 different regexes to handle them in the initial stages
    3. Words are separated by the standard <space> and <\n>
    4. If it ends a line without a <\n> or a punctuation
    5. If it ends with a punctuation
    Once we have the initial set of words
    6. We remove the punctations from the starting and end of words and strip them of white spaces and finally get our words that we need
    """
    tokenizer = RegexpTokenizer('[^ \n]+[ \n]|[A-Za-z0-9-]+$|[A-Za-z0-9-]+[.?!\n]')
    newText = tokenizer.tokenize(text)
    for t in range(len(newText)):
        newText[t] = re.sub(r'[^a-zA-Z0-9]*$', "", newText[t])
        newText[t] = re.sub(r'^[^a-zA-Z0-9]*', "", newText[t])
    return newText

In [129]:
getWords(' All I can say is that I will not consume it')

['All', 'I', 'can', 'say', 'is', 'that', 'I', 'will', 'not', 'consume', 'it']

In [130]:
# TASK 1
def task1(filename):
    """
    We use the existing getSentences and getWords function
    Note: We are only considering words in the main text not the metadata
    For each sentence we use getWords() function to get words in that sentence and count them
    """
    text = readFile(filename)
    newSentences = getSentences(text)
    words = []
    lenWords = 0
    for s in newSentences:
        words = getWords(s)
        lenWords += len(words)
    print("Number of words: %d\nNumber of sentences: %d" % (lenWords, len(newSentences)))

In [138]:
# TASK 2
def task2(filename):
    """
    Like task1 we use the getSentences and getWords function to get all words for all sentences. 
    Then we check the first letter of the words in each sentence to see if it's a vowel 
    If it is we increase the count of vowels otherwise we increase count of consonants
    """
    text = readFile(filename)
    newSentences = getSentences(text)
    words = [] # Final list of list of words
    lenWords = 0
    for s in newSentences:
        words.append(getWords(s))
    vowels = []
    consonants = []
    vlist = ['a','e','i','o', 'u'] # List of vowels
    for word in words:
        for w in word:
            if (w[0].lower() in vlist): # w[0] signifies the first letter in a word
                vowels.append(w)
            else:
                consonants.append(w)
    print("Starting with vowel: %d\nStarting with consonants: %d" % (len(vowels), len(consonants)))

In [84]:
# TASK 3
def task3(filename):
    """
    We use a regex made by observing the alot of files
    We have also kept a scope of improvement in mind considering emails could be of numerous formats
    We use finditer function to find all the matching regex and then print 0th index
    Which stores the first found group which is also the entire match
    """
    text = readFile(filename)
    regex = re.finditer(r'[a-zA-Z0-9_.]+@[a-zA-Z0-9-]+([.][a-zA-Z]+)*', text)
    for r in regex:
        print(r[0])

In [85]:
# TASK 4
def task4(filename, W):
    """
    The following functions are very similar to task2
    We are using the numToWord library to convert number input to convert to string for comparison of wider horizon
    By iterating sentences and further the words we compare the first word of the sentence with the desired words 
    """
    wordsToCheck = [str(W)]
    if (str(W).isdecimal()):
        wordsToCheck.append(num2words.num2words(W))
    text = readFile(filename)
    sentences = getSentences(text)
    count = 0
    for word in wordsToCheck:
        for s in sentences:
            words = getWords(s)
            if (words[0].lower() == word.lower()):
                count += 1
                print(s)
    print("Total Number of sentences starting with %s are %d" % (word, count))

In [101]:
# TASK 5
def task5(filename, W):
    """
    The following functions are very similar to task2
    We are using the numToWord library to convert number input to convert to string for comparison of wider horizon
    By iterating sentences and further the words we compare the last word of the sentence with the desired words 
    """
    wordsToCheck = [str(W)]
    if (str(W).isdecimal()):
        wordsToCheck.append(num2words.num2words(W))
    text = readFile(filename)
    sentences = getSentences(text)
    count = 0
    for word in wordsToCheck:
        for s in sentences:
            words = getWords(s)
            if (words[-1].lower() == word.lower()):
                count += 1
                print(s)
    print("Total Number of sentences ending with %s are %d" % (word, count))

In [123]:
# TASK 6
def task6(filename, W):
    """
    The following functions are very similar to task2
    We are using the numToWord library to convert number input to convert to string for comparison of wider horizon
    By iterating sentences and further the words we compare each word of the sentence with the desired words to see if we have a match
    We also have a flag which prevents a same sentence being printed twice in case we have one sentence having 2 or more occurance of words
    """
    wordsToCheck = [str(W)]
    if (str(W).isdecimal()):
        wordsToCheck.append(num2words.num2words(W))
    text = readFile(filename)
    sentences = getSentences(text)
    count = 0
    for word in wordsToCheck:
        for s in sentences:
            words = getWords(s)
            flag = False # TO prevent same sentence from being printed twice
            for w in words:
                if (w.lower() == word.lower()):
                    count += 1
                    if (not flag):
                        print(s)
                        flag = True
    print("Total Count of %s in sentences is %d" % (word, count))

In [103]:
# TASK 7
def task7(filename):
    """
    For checking the question we have gone with the simple method of comparing question marks at the end of a sentence.
    This has been done to prevent from having multiple question marks being counted separately
    """
    text = readFile(filename)
    sentences = getSentences(text)
    count = 0
    for s in sentences:
        if (s[-1] == '?'):
            print(s)

In [104]:
# TASK 8
def task8(filename):
    """
    The time was in the format of 
    Date Month Year Hours:Minutes:Seconds TimeZone
    And that is what the regex des
    """
    text = readFile(filename)
    sentences = getSentences(text)
    count = 0
    for m in re.finditer(r"[0-9]+[ ][a-zA-Z]+[ ][0-9]+ ([0-9]+):([0-9]+):([0-9]+) [a-zA-Z]+", text):
        print("%s min, %s sec" % (m.group(2), m.group(3)))

In [120]:
# TASK 9
def task9(filename):
    """
    For case of abbreviatiosn we are considering a couple of cases
    1. Multiple capital letters together generally are abbreviations
    2. Multiple alphanumeric characters separated by multiple periods
    3. Alphas with having aleast one period between them
    """
    text = readFile(filename)
    sentences = getSentences(text)
    count = 0
    for m in re.finditer(r"(([A-Za-z]\.([A-Za-z]\.)+)|([A-Z][A-Z]+)|(\.[a-zA-Z]+\.))", text):
        print(m.group(0))

In [131]:
task1('sci.med/58841')


Number of words: 80
Number of sentences: 5


In [140]:
task1('sci.med/58921')

Number of words: 344
Number of sentences: 25


In [141]:
task2('sci.med/58921')

Starting with vowel: 122
Starting with consonants: 222


In [109]:
task3('sci.med/58921')

aldridge@netcom.com
aldridgeC5nL9z.H2z@netcom.com
1993Apr17.181013.3743@uvm.edu
hbloom@moose.uvm.edu


In [110]:
task4('sci.med/58921', 'I')

I don't drink it.
I figure sugar was made for a reason.
Total Number of sentences starting with I are 2


In [111]:
task5('sci.med/58921', 'it')

All I can say is that I will not consume it.
I don't drink it.
Total Number of sentences ending with it are 2


In [124]:
task6('sci.med/58921', 'it')

Some people are concerned about the chemicals that the  body produces 
>when it degrades nutrasweet.
It is thought to form formaldehyde and known to
>for methanol in the degredation pathway that the body uses to eliminate 
>substances.
All I can say is that I will not consume it.
It is an amino acid, and everyone uses small
>quantities of it for protein synthesis in the body.
Some people have a disease
>known as phenylketoneurea, and they are missing the enzyme necessary to 
>degrade this compound and eliminate it from the body.
For them, it will 
>accumulate in the body, and in high levels this is toxic to growing nerve
>cells.
Therefore, it is Only a major problem in young children (until around
>age 10 or so) or women who are pregnant and have this disorder.
It used to
>be a leading cause of brain damage in infants, but now it can be easily 
>detected at birth, and then one must simply avoid comsumption of phenylalanine
>as a child, or when pregnant.
And those
who only have one gene

In [117]:
task7('sci.med/58921')

If I remember rightly PKU syndrome in infants is about 1/1200 ?
And people who lack one gene are supposed to be 1/56 persons?
The methol, formaldahyde thing was supposed to occur with heating?


In [118]:
task8('sci.med/58921')

27 min, 35 sec


In [121]:
task9('sci.med/58921')

.srv.
.cmu.
.harvard.
.near.
.reston.
.american.
.peachnet.
ID
NETCOM
GMT
.uvm.
PKU
PKU
